In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
data = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')

In [4]:
len(data)

1154112

In [5]:
data.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [6]:
data['trip_dur'] = (data['dropOff_datetime'] -  data['pickup_datetime']).astype('timedelta64[s]')/60

In [7]:
data['trip_dur'].mean()

19.1672240937939

In [8]:
data.head(10)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_dur
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000
6,B00037,2021-01-01 00:18:12,2021-01-01 00:30:04,NaN,91.0,None,B00037,11.866667
7,B00037,2021-01-01 00:36:15,2021-01-01 00:45:08,NaN,39.0,None,B00037,8.883333
8,B00037,2021-01-01 00:55:04,2021-01-01 01:13:02,NaN,37.0,None,B00037,17.966667
9,B00037,2021-01-01 00:48:40,2021-01-01 01:12:02,NaN,39.0,None,B00037,23.366667


In [9]:
data = data[(data.trip_dur>=1) & (data.trip_dur<=60)]

In [10]:
len(data)

1109826

In [11]:
data['PUlocationID'] = data['PUlocationID'].fillna(-1)
data['DOlocationID'] = data['DOlocationID'].fillna(-1)

In [12]:
(data['PUlocationID'] == -1).sum()/len(data)*100

83.52732770722618

In [13]:
categorical = ['PUlocationID', 'DOlocationID']
target = ['trip_dur']

In [14]:
dv = DictVectorizer()

In [15]:
data[categorical] = data[categorical].astype(str)

In [16]:
train_dicts = data[categorical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
y_train = data[target].values

In [17]:
X_train.shape

(1109826, 525)

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.5285191072072

In [19]:
data1 = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')

In [20]:
data1.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037


In [21]:
data1['trip_dur'] = (data1['dropOff_datetime'] -  data1['pickup_datetime']).astype('timedelta64[s]')/60

In [22]:
data1.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_dur
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014,92.000000
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021,7.950000
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037,13.800000


In [23]:
data1 = data1[(data1.trip_dur>=1) & (data1.trip_dur<=60)]
data1 = data1[data1.DOlocationID!=225.0]
data1['PUlocationID'] = data1['PUlocationID'].fillna(-1)
data1['DOlocationID'] = data1['DOlocationID'].fillna(-1)

In [24]:
data1.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_dur
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021,7.950000
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1.0,61.0,None,B00037,8.966667
6,B00112,2021-02-01 00:30:25,2021-02-01 00:57:23,-1.0,26.0,None,B00112,26.966667


In [25]:
data1[categorical] = data1[categorical].astype(str)
val_dicts = data1[categorical].to_dict(orient='records')

#dv = DictVectorizer()
X_val = dv.fit_transform(val_dicts)
y_val = data1[target].values

In [26]:
len(dv.feature_names_)

525

In [27]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.7815149939228